In [ ]:
import os
from huggingface_hub import HfApi, create_repo, list_repo_files
from huggingface_hub.utils import RepositoryNotFoundError, HfHubHTTPError
from logbar import LogBar

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

log = LogBar.shared()
api = HfApi()


def sync_model_to_hf(local_path: str, repo_id: str, private: bool = False):
    """Ensure Hugging Face repo exists, delete old files, and upload new model files."""
    log.info(f"Syncing local '{local_path}' -> hub:{repo_id}")
    
    # Ensure repo exists; create if not
    try:
        list_repo_files(repo_id=repo_id, repo_type="model")
    except RepositoryNotFoundError:
        log.info(f"Repo not found: creating {repo_id}")
        create_repo(repo_id=repo_id, repo_type="model", private=private, exist_ok=True)
    except HfHubHTTPError as e:
        log.error(f"Cannot access {repo_id}: HTTP error {e}")
        return
    except Exception as e:
        log.error(f"Unexpected error checking repo '{repo_id}': {e}")
        return

    # Delete all existing files
    try:
        files = list_repo_files(repo_id=repo_id, repo_type="model")
        for file in files:
            log.info(f"Deleting '{file}' from {repo_id}")
            api.delete_file(path_in_repo=file, repo_id=repo_id, repo_type="model")
    except Exception as e:
        log.error(f"Failed to delete files in {repo_id}: {e}")
        return

    # Upload new content
    try:
        log.info(f"Uploading local folder '{local_path}' to {repo_id}")
        api.upload_folder(folder_path=local_path, repo_id=repo_id, repo_type="model")
    except Exception as e:
        log.error(f"Upload failed for {repo_id}: {e}")
        return

    log.info(f"Successfully synced '{local_path}' -> {repo_id}")


sync_model_to_hf(
    local_path="/scratch/izar/delsad/mnlp_cache/EfficientQAT/torch/Qwen3-w4g64-20",
    repo_id="TheS3b/Qwen3-EfficientQAT-w4g64-20",
    private=False,
)
sync_model_to_hf(
    local_path="/scratch/izar/delsad/mnlp_cache/EfficientQAT/gptq/Qwen3-w2g64-20",
    repo_id="TheS3b/Qwen3-EfficientQAT-w2g64-20",
    private=False,
)


INFO  Syncing local '/scratch/izar/delsad/mnlp_cache/EfficientQAT/torch/Qwen3-w4g64' -> hub:TheS3b/Qwen3-EfficientQAT-w4g64-torch
INFO  Repo not found: creating TheS3b/Qwen3-EfficientQAT-w4g64-torch           
INFO  Deleting '.gitattributes' from TheS3b/Qwen3-EfficientQAT-w4g64-torch     
INFO  Uploading local folder '/scratch/izar/delsad/mnlp_cache/EfficientQAT/torch/Qwen3-w4g64' to TheS3b/Qwen3-EfficientQAT-w4g64-torch


model.safetensors:   0%|          | 0.00/550M [00:00<?, ?B/s]
model.safetensors:   1%|▏         | 7.13M/550M [00:00<00:07, 69.3MB/s]
tokenizer.json: 100%|██████████| 11.4M/11.4M [00:00<00:00, 11.8MB/s]s]
model.safetensors: 100%|██████████| 550M/550M [00:14<00:00, 39.2MB/s] 
Upload 2 LFS files: 100%|██████████| 2/2 [00:14<00:00,  7.13s/it]


INFO  Successfully synced '/scratch/izar/delsad/mnlp_cache/EfficientQAT/torch/Qwen3-w4g64' -> TheS3b/Qwen3-EfficientQAT-w4g64-torch
